# 03. News classification with BERT

Author: [ArtyomR](https://github.com/ArtyomR)  
Idea and big part of source code: [Konstantin Shitkov](https://github.com/shitkov)  
Source:

<span style="color:red">This Notebook was tested in Google Colab with GPU.</span>

**In this Notebook I'll walk through News classification with BERT**  
Next steps were implemented.
1. Text preparation, deviding train dataset on train and validation parts
2. BERT classifire initialisation and training
3. Check test data and scoring

Dataset was creted within [Chinese AI news agregation project](https://t.me/chinese_ai_news).  

## Get data from github

In [1]:
!wget https://raw.githubusercontent.com/ArtyomR/AI-News-Classification/main/bert_classifier.py
!wget https://raw.githubusercontent.com/ArtyomR/AI-News-Classification/main/bert_dataset.py
!wget https://raw.githubusercontent.com/ArtyomR/AI-News-Classification/main/ai_articles_test_231124.xlsx
!wget https://raw.githubusercontent.com/ArtyomR/AI-News-Classification/main/ai_articles_train_231124.xlsx

--2023-12-22 12:56:49--  https://raw.githubusercontent.com/ArtyomR/AI-News-Classification/main/bert_classifier.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5278 (5.2K) [text/plain]
Saving to: ‘bert_classifier.py’

bert_classifier.py  100%[===================>]   5.15K  --.-KB/s    in 0s      

2023-12-22 12:56:49 (55.5 MB/s) - ‘bert_classifier.py’ saved [5278/5278]

--2023-12-22 12:56:49--  https://raw.githubusercontent.com/ArtyomR/AI-News-Classification/main/bert_dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89

## Installing Packages

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
from bert_dataset import CustomDataset      # Custom Datase class fron bert_dataset.py
from bert_classifier import BertClassifier    # BERT classifier class from bert_classifier.py

In [4]:
train_valid_data = pd.read_excel('/content/ai_articles_train_231124.xlsx', index_col=0)
train_valid_data.head(2)

,published_flag,combined_text_ru,combined_text_ru_clean
723,1,"Школьный гимн и музыка на электрогитаре, котор...",школьный гимн и музыка на электрогитара которы...
2997,0,Недавний минимум в 185 юаней: самостоятельная ...,недавний минимум в 185 юань самостоятельный по...


In [5]:
test_data  = pd.read_excel('/content/ai_articles_test_231124.xlsx',index_col=0)
test_data.head(2)

,published_flag,combined_text_ru,combined_text_ru_clean
1167,1,Tencent Cloud запускает продукт для рисования ...,tencent cloud запускать продукт для рисование ...
1542,1,Генеративный ИИ достиг «пика завышенных ожидан...,генеративный ия достигнуть пик завысить ожидан...


In [8]:
# split dtaset on train and valid_data dataframes
train_data, valid_data = train_test_split(
    train_valid_data,
    test_size=0.2,
    random_state=72
)
print(len(train_data),'\n',train_data.head(2))
print(len(valid_data),'\n',valid_data.head(2))

1726 
       published_flag                                   combined_text_ru  \
603                1  Китай активизирует усилия по созданию чипов ис...   
2621               1  Первая партия Xiaoice AI-клонов интернет-знаме...   

                                 combined_text_ru_clean  
603   китай активизировать усилие по создание чип ис...  
2621  первый партия xiaoice aiклон интернетзнаменито...  
432 
       published_flag                                   combined_text_ru  \
941                1  Самый богатый человек Гонконга Ли Ка-шинг поже...   
2511               1  Может ли ИИ дать Китаю преимущество, чтобы пре...   

                                 combined_text_ru_clean  
941   самый богатый человек гонконг ли кашинга пожер...  
2511  мочь ли ия дать китай преимущество чтобы превз...  


## Initialize BERT classifier

In [9]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny2',
        tokenizer_path='cointegrated/rubert-tiny2',
        n_classes=2,
        epochs=10,
        model_save_path='/content/bert.pt'
)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

Prepare data and helpers for train and evlauation

In [10]:
classifier.preparation(
        X_train=list(train_data['combined_text_ru']),
        y_train=list(train_data['published_flag']),
        X_valid=list(valid_data['combined_text_ru']),
        y_valid=list(valid_data['published_flag'])
    )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Train loop

In [11]:
%%time
classifier.train()

Epoch 1/10
Train loss 0.7064334983885642 accuracy 0.7421784472769409
Val loss 0.7703815704153385 accuracy 0.7754629629629629
----------
Epoch 2/10
Train loss 0.601434906167024 accuracy 0.8256083429895712
Val loss 0.896010055582371 accuracy 0.787037037037037
----------
Epoch 3/10
Train loss 0.477625686663473 accuracy 0.873117033603708
Val loss 0.9465950658801981 accuracy 0.7893518518518519
----------
Epoch 4/10
Train loss 0.3723681193518204 accuracy 0.9049826187717265
Val loss 1.0605745652402996 accuracy 0.7916666666666666
----------
Epoch 5/10
Train loss 0.2769390861650479 accuracy 0.9409038238702202
Val loss 1.179895810488031 accuracy 0.7893518518518519
----------
Epoch 6/10
Train loss 0.18815286698377018 accuracy 0.9600231749710313
Val loss 1.248123752758046 accuracy 0.7916666666666666
----------
Epoch 7/10
Train loss 0.15413071049711966 accuracy 0.966396292004635
Val loss 1.3134435564402844 accuracy 0.7916666666666666
----------
Epoch 8/10
Train loss 0.12559444535681516 accuracy 0.9

## Check test data

In [12]:
%%time
texts = list(test_data['combined_text_ru'])
labels = list(test_data['published_flag'])

predictions = [classifier.predict(t) for t in texts]

CPU times: user 4.99 s, sys: 23 ms, total: 5.01 s
Wall time: 5.04 s


In [13]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.8227945568580601, recall: 0.828564303526897, f1score: 0.8229331268659781


In [14]:
from sklearn.metrics import accuracy_score, classification_report

In [15]:
# Model accuracy
accuracy = accuracy_score(labels, predictions)
print(f'Accuracy: {accuracy}')

# Additional metrics
print(classification_report(labels, predictions))

Accuracy: 0.8239740820734341
              precision    recall  f1-score   support

           0       0.88      0.79      0.84       525
           1       0.76      0.86      0.81       401

    accuracy                           0.82       926
   macro avg       0.82      0.83      0.82       926
weighted avg       0.83      0.82      0.82       926



**10 epochs**  
Accuracy: 0.8196544276457883  
              precision    recall  f1-score   support

           0       0.84      0.85      0.84       525
           1       0.80      0.78      0.79       401

    accuracy                           0.82       926
   macro avg       0.82      0.82      0.82       926  
weighted avg       0.82      0.82      0.82       926


**2 epochs**  
Accuracy: 0.7980561555075594  
    precision    recall  f1-score   support

           0       0.83      0.81      0.82       525
           1       0.76      0.79      0.77       401

    accuracy                           0.80       926
   macro avg       0.79      0.80      0.80       926  
weighted avg       0.80      0.80      0.80       926





In [16]:
type(classifier)

bert_classifier.BertClassifier

## References
- [BERT для классификации русскоязычных текстов](https://habr.com/ru/articles/567028/)
- [News Article Classification Task using SOTA models and their comparison](https://medium.com/mlearning-ai/news-article-classification-task-using-sota-models-and-their-comparison-a0f87abdc0c1)